# 자계추 hw1: Create Dataset

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

## Load Datasets

In [89]:
CWD = Path('.').resolve()
DATA_DIR = CWD / 'data'

In [90]:
CRSP_M_df = pd.read_csv(DATA_DIR / 'CRSP_M.csv')
compustat_df = pd.read_csv(DATA_DIR / 'compustat_permno.csv') 
sample_df = pd.read_csv(DATA_DIR / 'assignment1_sample_data.csv')

## SAS3

Construct BE Data

Compustat 데이터 사용

fiscal year 별로 되어있음. 

In [91]:
# * Calculate BE; 
# data BE; 
#  set compustat_permno (where = (missing(permno) = 0)); 

compustat_df.dropna(subset=['permno'], inplace=True)

In [92]:
# NOTE: The data set WORK.BE has 264450 observations and 6 variables.

compustat_df.shape 

(264450, 10)

In [93]:
#  year = year(datadate); 

compustat_df['year'] = compustat_df['datadate'] // 10000

In [94]:
# if missing(ITCB) then ITCB = 0; * investment tax credit; 

compustat_df['itcb'] = compustat_df['itcb'].fillna(0)

In [95]:
# SAS 코드에는 없는 내용. 하지만 확인해보면 preferred stock redemption value가 음수인 경우가 있음.
# 일단 원본 코드에 없으므로 무시하고 넘어감.

# compustat_df.loc[
#     compustat_df['pstkrv'] < 0,
#     'pstkrv'
#     ] = 0

In [96]:
# BVPS = PSTKRV; * bool value of preferred stock (BVPS) = preferred stock 의 redemption value로 일단 놓고; 
#  if missing(BVPS) then BVPS = PSTKL; * 없으면 preferred stock 의 liquidating value; 
#  if missing(BVPS) then BVPS = PSTK; * 또 없으면 preferred stock의 par value; 
#  if missing(BVPS) then BVPS = 0; * 다 없으면 0;

compustat_df['bvps'] = compustat_df['pstkrv'].fillna(compustat_df['pstkl']) \
                                               .fillna(compustat_df['pstk']) \
                                                .fillna(0)

In [97]:
# BE = SEQ + TXDB + ITCB - BVPS; * If SEQ or TXDB is missing, BE, too, will be missing; 
#  if BE<=0 then BE = .; * If BE<0, the value of BE is taken to be missing;  

compustat_df['be'] = compustat_df['seq'] \
                    + compustat_df['txdb'] \
                    + compustat_df['itcb'] \
                    - compustat_df['bvps']

compustat_df.loc[compustat_df['be'] <= 0, 'be'] = np.nan

In [98]:
# * In some cases, firms change the month in which their fiscal year ends,  
# * resulting in two entries in the Compustat database for the same calendar year y.  
# * In such cases, data from the latest in the given calendar year y are used.;  
# proc sort data = BE; by gvkey permno year datadate; run; 
# data BE; 
#  set BE; 
#  by gvkey permno year datadate; 
#  if last.year; 
# run; 
# proc sort data = BE nodupkey; by gvkey permno year datadate; run;

compustat_df.sort_values(['gvkey', 'permno', 'year', 'datadate'], inplace=True)

compustat_df['idx'] = compustat_df.index

groupbylast = compustat_df.groupby(['gvkey', 'permno', 'year']).last().reset_index()
dropduplicates = compustat_df.drop_duplicates(subset=['gvkey', 'permno', 'year'], keep='last').reset_index()

# compustat_df = compustat_df.groupby(['gvkey', 'permno', 'year', ]).last().reset_index()
compustat_df.drop_duplicates(subset=['gvkey', 'permno', 'year'], inplace=True, keep='last')

# 이 두가지의 결과가 다르다. 당장 row 수는 같게 나오는데 나중에 결과가 다르다. 

In [99]:
groupbylast['idx']

0              9
1             10
2             11
3             12
4             13
           ...  
263849    434210
263850    434218
263851    434223
263852    434228
263853    434229
Name: idx, Length: 263854, dtype: int64

In [100]:
dropduplicates['idx']

0              9
1             10
2             11
3             12
4             13
           ...  
263849    434210
263850    434218
263851    434223
263852    434228
263853    434229
Name: idx, Length: 263854, dtype: int64

In [102]:
(groupbylast['idx'] == dropduplicates['idx']).all()

True

??? 심지어 원래의 index를 비교해도 같다? 그럼 정말 둘이 같다는 소리인데... 

In [12]:
compustat_df

,gvkey,datadate,itcb,pstk,pstkl,pstkrv,seq,txdb,permno,permco,year,bvps,be
9,1000,19701231,0.0,0.000,0.000,0.000,10.544,0.000,25881.0,23369.0,1970,0.000,10.544
10,1000,19711231,0.0,0.000,0.000,0.000,8.382,0.000,25881.0,23369.0,1971,0.000,8.382
11,1000,19721231,0.0,0.000,0.000,0.000,7.021,0.288,25881.0,23369.0,1972,0.000,7.309
12,1000,19731231,0.0,0.000,0.000,0.000,8.567,0.231,25881.0,23369.0,1973,0.000,8.798
13,1000,19741231,0.0,0.414,0.414,2.069,10.257,0.091,25881.0,23369.0,1974,2.069,8.279
...,...,...,...,...,...,...,...,...,...,...,...,...,...
434210,296318,20121231,0.0,0.000,0.000,0.000,2908.515,0.000,13013.0,53885.0,2012,0.000,2908.515
434218,296753,20121231,0.0,0.000,0.000,0.000,-22.570,0.000,13255.0,53991.0,2012,0.000,NaN
434223,296885,20121231,0.0,0.000,0.000,0.000,19.018,0.000,13707.0,54281.0,2012,0.000,19.018
434228,297209,20111231,0.0,0.000,0.000,0.000,2274.073,NaN,13104.0,53928.0,2011,0.000,NaN


In [13]:
# NOTE: The data set WORK.BE has 263854 observations and 6 variables.

compustat_df.shape

(263854, 13)

## SAS5

Construct ME and return data

CRSP 데이터 사용

Monthly data로 되어있음 

In [14]:
# * SAS 4: Merge CRSP stock and event file and add risk-free rate *******************; 

# %let filter=%str(shrcd in (10,11) and exchcd in (1,31,2,32,3,33)); 

# %crspmerge(s = m, outset = CRSP, 
# start = &start_date, end = &end_date, 
# sfvars = permco ret vol shrout prc altprc, 
# sevars = siccd shrcd exchcd dlstcd dlret, 
# filters=&filter);   


# filters # 사실 필터는 이미 적용되어 있음
filter_common_stocks = [10, 11] # SHRCD
filter_exchange = [ # EXCHCD
    1, 31, # NYSE
    2, 32, # AMEX
    3, 33, # NASDAQ
]

CRSP_M_df = CRSP_M_df[ CRSP_M_df['SHRCD'].isin(filter_common_stocks) ]
CRSP_M_df = CRSP_M_df[ CRSP_M_df['EXCHCD'].isin(filter_exchange) ]

In [15]:
CRSP_M_df

,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf
0,19610131,NaN,10006,10,1,3740.0,NaN,22156,50.25,939.0,0.322368,1420.0,50.2500,0.0019
1,19610131,NaN,10014,10,1,3710.0,NaN,22157,4.00,395.0,0.000000,2504.0,4.0000,0.0019
2,19610131,NaN,10030,10,1,3310.0,NaN,22160,41.75,280.0,0.087948,1627.0,41.7500,0.0019
3,19610131,NaN,10057,11,1,3540.0,NaN,20020,54.00,152.0,0.142857,500.0,54.0000,0.0019
4,19610131,NaN,10102,10,1,2810.0,NaN,22164,79.50,480.0,0.032468,3965.0,79.5000,0.0019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921188,20121231,574.0,76999,11,3,7372.0,-0.765517,11056,NaN,123365.0,NaN,6855.0,0.3120,0.0001
2921189,20121231,580.0,93007,11,3,9999.0,-0.774834,53201,NaN,121619.0,NaN,57097.0,0.6307,0.0001
2921190,20121231,584.0,38790,11,2,1311.0,-0.762470,1933,NaN,21350.0,NaN,19048.0,0.3321,0.0001
2921191,20121231,584.0,89761,11,2,3714.0,2.520000,44123,NaN,39636.0,NaN,7107.0,0.3700,0.0001


In [16]:
CRSP_M_df.shape # NOTE: The data set WORK.CRSP has 2921193 observations and 13 variables.

(2921193, 14)

In [17]:
# * SAS 5: Construct ME and return data *************************************; 

# * Calculate excess return adjusted for delising; 
# data CRSP_M2; 
#  set CRSP_M; 
#  year = year(date); *** date, 매달 마지막 거래일 값이 들어가있다. 거기서 연도를 뽑아냄. ;

CRSP_M_df['YEAR'] = CRSP_M_df['DATE'] // 10000

In [18]:
# * calculate market capitalization; 
#  if abs(altprc)>0 and shrout>0 then Meq = abs(altprc)*shrout/1000;  
# *** 절대값이 0보다 크면, market equity 값을 계산을 해라. (Meq) 만족 안하면 missing으로 처리.;
# ** ALTPRC: last non-missing price over all days in the month인데,  ;
# ** CRSP는 거래가 없을 경우 last bid와 last ask의 평균을 - 로 report함. ;
# ** 즉, 가격이 -인 것이 오류가 아니라는 소리임. 날려버리면 안됨. ; 
# ** 진짜 데이터가 available하지 않은 경우는 0이나 missing으로 표시해줌. ;

CRSP_M_df['MEQ'] = np.nan
CRSP_M_df.loc[ 
    (CRSP_M_df['ALTPRC'].abs() > 0) & (CRSP_M_df['SHROUT'] > 0) , 
    'MEQ'
    ] = CRSP_M_df['ALTPRC'].abs() * CRSP_M_df['SHROUT'] / 1000

In [19]:
# * if dlret is missing, follow Shumway (1997) to determine dlret; 
#  if missing(dlstcd) = 0 and missing(dlret) =1 then do; *** delisting code(사유)는 있고 delisting return이 missing이면, 아래와 같이 처리.;
#   if dlstcd in (500, 520, 574, 580, 584) or (dlstcd>=551 and dlstcd<=573)  
#    then dlret = -0.3; *** 위 사유들에 대해선 적당히 -0.3으로 처리;
#   else dlret = -1; *** 그 외에는 -1 (-100%)로 처리;
#  end; 

dlstcd_filter = [500, 520, 574, 580, 584] + list(range(551, 573+1))
CRSP_M_df.loc[
    (CRSP_M_df['DLSTCD'].isin(dlstcd_filter)) & 
    (CRSP_M_df['DLRET'].isna()), 
    'DLRET'
    ] = -0.3

CRSP_M_df.loc[
    (~CRSP_M_df['DLSTCD'].isin(dlstcd_filter)) & \
    CRSP_M_df['DLSTCD'].notna() & \
    (CRSP_M_df['DLRET'].isna()), \
    'DLRET'
    ] = CRSP_M_df['DLRET'].fillna(-1)

In [20]:
#  * calculate return adjusted for delisting; 
#  if missing(dlstcd) = 0 then do; 
#   if missing(ret) = 0 then retadj = (1+ret)*(1+dlret)-1; 
#   else retadj = dlret; 
#  end; 
#  else retadj = ret; 
#  eretadj = retadj - rf; *** 이게 최종적으로 사용하는 return. risk-free rate를 빼준 것. ;
# run;
# proc sort data = CRSP_M2; by date permco Meq; run; 

# dlstcd가 있을 때 
CRSP_M_df.loc[ # delisting 날의 ret가 있으면 (1+ret)*(1+dlret)-1
    CRSP_M_df['DLSTCD'].notna() & CRSP_M_df['RET'].notna(),
    'RETADJ'
    ] = (1 + CRSP_M_df['RET']) * (1 + CRSP_M_df['DLRET']) - 1

CRSP_M_df.loc[ # delisting 날의 ret가 없으면 dlret
    CRSP_M_df['DLSTCD'].notna() & CRSP_M_df['RET'].isna(),
    'RETADJ'
    ] = CRSP_M_df['DLRET']

# dlstcd가 없을 때
CRSP_M_df.loc[
    CRSP_M_df['DLSTCD'].isna(),
    'RETADJ'
    ] = CRSP_M_df['RET']

CRSP_M_df['ERETADJ'] = CRSP_M_df['RETADJ'] - CRSP_M_df['rf']
CRSP_M_df.sort_values(['DATE', 'PERMCO', 'MEQ'], inplace=True)

In [21]:
CRSP_M_df

,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf,YEAR,MEQ,RETADJ,ERETADJ
357,19610131,NaN,17670,10,1,2080.0,NaN,74,32.00,159.0,0.075000,955.0,32.00,0.0019,1961,30.56000,0.075000,0.073100
416,19610131,NaN,18702,10,1,5810.0,NaN,267,35.00,225.0,-0.075908,1587.0,35.00,0.0019,1961,55.54500,-0.075908,-0.077808
1086,19610131,NaN,68523,11,1,3830.0,NaN,301,95.00,860.0,0.183801,2476.0,95.00,0.0019,1961,235.22000,0.183801,0.181901
541,19610131,NaN,20714,10,1,5610.0,NaN,584,19.75,119.0,0.025974,1688.0,19.75,0.0019,1961,33.33800,0.025974,0.024074
67,19610131,NaN,11287,10,1,3740.0,NaN,921,14.25,346.0,0.117647,1760.0,14.25,0.0019,1961,25.08000,0.117647,0.115747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918038,20121231,NaN,13730,11,1,2911.0,NaN,54291,29.05,239843.0,NaN,20500.0,29.05,0.0001,2012,595.52500,NaN,NaN
2918022,20121231,NaN,13656,11,3,9999.0,NaN,54517,32.56,21929.0,0.032667,23100.0,32.56,0.0001,2012,752.13600,0.032667,0.032567
2918081,20121231,NaN,16852,11,3,3670.0,NaN,56028,5.28,105896.0,0.900649,1511.0,5.28,0.0001,2012,7.97808,0.900649,0.900549
2918086,20121231,NaN,17279,11,1,2111.0,NaN,56249,116.67,196351.0,-0.037058,129445.0,116.67,0.0001,2012,15102.34815,-0.037058,-0.037158


In [22]:
CRSP_M_df.shape # NOTE: The data set WORK.CRSP_M2 has 2921193 observations and 18 variables.

(2921193, 18)

In [23]:
# * There are cases when the same firm (permco) has two or more securities (permno)  
# at the same date.  
# * We aggregate all ME for a given permco and date,       
# * and assign this aggregated ME to the permno with the largest ME; 
# data CRSP_M3; 
#  set CRSP_M2; 
#  by date permco Meq; 
#  retain ME;  
#  if first.permco and last.permco then do; 
#   ME = Meq; *** Meq는 각 share class의 Market equity, ME는 각 회사(permco)의 Market equity의 합. ;
#   output; 
#  end; 


# nan 포함하여 groupby 하여 개수 확인
CRSP_M_df['count_permno'] = CRSP_M_df.groupby(['DATE', 'PERMCO'])['PERMNO'].transform('size')

# ME를 일단 nan으로 초기화
CRSP_M_df['ME'] = np.nan

# first.permco and last.permco 즉 1개인 경우
CRSP_M_df.loc[CRSP_M_df['count_permno'] == 1, 'ME'] = CRSP_M_df['MEQ']


In [24]:
#  else do; 
#   if first.permco then ME = Meq; *** ME는 Meq의 누적합. ;
#   else ME = sum(Meq, ME); *** 누적합하는 컬럼 ME를 만들었으니, 누적합하는데 사용한 그 이전의 row들은 다 날림. ;
#   If last.permco then output; 
#  end; 

# 2개 이상인 경우, MEQ를 합
CRSP_M_df.loc[CRSP_M_df['count_permno'] > 1, 'ME'] = CRSP_M_df.groupby(['DATE', 'PERMCO'])['MEQ'].transform('sum')

# 가장 큰 MEQ를 가진 PERMNO를 선택
CRSP_M_df = CRSP_M_df.sort_values(['DATE', 'PERMCO', 'MEQ'], ascending=[True, True, True])
CRSP_M_df = CRSP_M_df.groupby(['DATE', 'PERMCO']).last().reset_index()

# 임시 컬럼 제거
CRSP_M_df.drop(columns=['count_permno'], inplace=True)

In [25]:
CRSP_M_df

,DATE,PERMCO,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PRC,VOL,RET,SHROUT,ALTPRC,rf,YEAR,MEQ,RETADJ,ERETADJ,ME
0,19610131,74,NaN,17670,10,1,2080.0,NaN,32.00,159.0,0.075000,955.0,32.00,0.0019,1961,30.56000,0.075000,0.073100,30.56000
1,19610131,267,NaN,18702,10,1,5810.0,NaN,35.00,225.0,-0.075908,1587.0,35.00,0.0019,1961,55.54500,-0.075908,-0.077808,55.54500
2,19610131,301,NaN,68523,11,1,3830.0,NaN,95.00,860.0,0.183801,2476.0,95.00,0.0019,1961,235.22000,0.183801,0.181901,235.22000
3,19610131,584,NaN,20714,10,1,5610.0,NaN,19.75,119.0,0.025974,1688.0,19.75,0.0019,1961,33.33800,0.025974,0.024074,33.33800
4,19610131,921,NaN,11287,10,1,3740.0,NaN,14.25,346.0,0.117647,1760.0,14.25,0.0019,1961,25.08000,0.117647,0.115747,25.08000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892460,20121231,54291,NaN,13730,11,1,2911.0,NaN,29.05,239843.0,NaN,20500.0,29.05,0.0001,2012,595.52500,NaN,NaN,595.52500
2892461,20121231,54517,NaN,13656,11,3,9999.0,NaN,32.56,21929.0,0.032667,23100.0,32.56,0.0001,2012,752.13600,0.032667,0.032567,752.13600
2892462,20121231,56028,NaN,16852,11,3,3670.0,NaN,5.28,105896.0,0.900649,1511.0,5.28,0.0001,2012,7.97808,0.900649,0.900549,7.97808
2892463,20121231,56249,NaN,17279,11,1,2111.0,NaN,116.67,196351.0,-0.037058,129445.0,116.67,0.0001,2012,15102.34815,-0.037058,-0.037158,15102.34815


In [26]:
CRSP_M_df.shape # NOTE: The data set WORK.CRSP_M3 has 2892465 observations and 19 variables.

(2892465, 19)

## SAS6

Merge BE and ME with return data

In [27]:
# proc sort data = crsp_m3 nodupkey; by permno date; run; *** duplicates 있는지 확인하려고 매번 체크하는 부분; 

# * SAS 6: Merge BE and ME with Return Data *************************************; 

# * Calculate BM from the previous year and June ME from this year for each permno; 
# data ME_Jun; 
#  set CRSP_M3 (where = (month(date) = 6 & missing(ME) = 0)); 
#  t = year(date); ** 1999 Dec ME --> t=2000 다음 해에 trading signal로 쓰도록. ; 
#  ME_Jun = ME; 
#  keep permno t ME_Jun; ** 이것들만 남기고 나머지는 버려라. ;
# run; 
CRSP_ME_JUN_df = CRSP_M_df.copy()
CRSP_ME_JUN_df['T'] = CRSP_ME_JUN_df['DATE'] // 10000
CRSP_ME_JUN_df.loc[
    (CRSP_ME_JUN_df['DATE'] % 10000 // 100 == 6 ) & \
    CRSP_ME_JUN_df['ME'].notna(), 
    'ME_JUN'
] = CRSP_ME_JUN_df['ME']

CRSP_ME_JUN_df = CRSP_ME_JUN_df[['PERMNO', 'T', 'ME_JUN',]]
CRSP_ME_JUN_df.sort_values(['PERMNO', 'T'], inplace=True)

In [28]:
CRSP_ME_JUN_df.dropna(subset=['ME_JUN'], inplace=True)

In [29]:
CRSP_ME_JUN_df

,PERMNO,T,ME_JUN
1112263,10000,1986,11.734594
1183863,10000,1987,0.851594
1112264,10001,1986,6.033125
1183864,10001,1987,5.822125
1258733,10001,1988,6.200000
...,...,...,...
2778355,93435,2010,102.653400
2825178,93435,2011,28.538400
2778356,93436,2010,2220.908340
2825179,93436,2011,3028.966530


In [30]:
CRSP_ME_JUN_df.shape # NOTE: There were 239521 observations read from the data set WORK.ME_JUN.

(239521, 3)

In [31]:
# data ME_last_Dec; 
#  set CRSP_M3 (where = (month(date) = 12 & missing(ME) = 0)); 
#  t = year(date)+1; ** 마찬가지로. +1 해준다. ;  
#  ME_last_Dec = ME; 
#  keep permno t ME_last_Dec; 
# run; 
# proc sort data = ME_last_Dec; by permno t; run; 

CRSP_ME_LAST_DEC_df = CRSP_M_df.copy()
CRSP_ME_LAST_DEC_df['T'] = CRSP_ME_LAST_DEC_df['DATE'] // 10000 + 1
CRSP_ME_LAST_DEC_df.loc[
    (CRSP_ME_LAST_DEC_df['DATE'] % 10000 // 100 == 12 ) & \
    CRSP_ME_LAST_DEC_df['ME'].notna(), 
    'ME_LAST_DEC'
] = CRSP_ME_LAST_DEC_df['ME']

CRSP_ME_LAST_DEC_df = CRSP_ME_LAST_DEC_df[['PERMNO', 'T', 'ME_LAST_DEC',]]
CRSP_ME_LAST_DEC_df.sort_values(['PERMNO', 'T'], inplace=True)

In [32]:
CRSP_ME_LAST_DEC_df.dropna(subset=['ME_LAST_DEC'], inplace=True)

In [33]:
CRSP_ME_LAST_DEC_df

,PERMNO,T,ME_LAST_DEC
1147621,10000,1987,1.981566
1147622,10001,1987,6.937000
1221511,10001,1988,5.828000
1295310,10001,1989,6.362250
1365986,10001,1990,10.347750
...,...,...,...
2802015,93435,2011,155.017720
2847854,93435,2012,23.864000
2802016,93436,2011,2527.400040
2847855,93436,2012,2987.490240


In [34]:
CRSP_ME_LAST_DEC_df.shape # NOTE: There were 242805 observations read from the data set WORK.ME_LAST_DEC.

(242805, 3)

In [35]:
# data BE_last_year; 
#  set BE (where = (missing(BE) = 0)); 
#  t = year+1; 
#  BE_last_year = BE; 
#  keep permno t BE_last_year; 
# run; 
# proc sort data = BE_last_year; by permno t; run;

compustat_be_last_year_df = compustat_df.copy()
compustat_be_last_year_df['t'] = compustat_be_last_year_df['year'] + 1
# compustat_be_last_year_df.dropna(subset=['be'], inplace=True)
compustat_be_last_year_df.loc[
    compustat_be_last_year_df['be'].notna(),
    'be_last_year'
    ] = compustat_be_last_year_df['be']

compustat_be_last_year_df = compustat_be_last_year_df[['permno', 't', 'be_last_year',]]
compustat_be_last_year_df.sort_values(['permno', 't'], inplace=True)
compustat_be_last_year_df.dropna(subset=['be_last_year'], inplace=True)

In [36]:
compustat_be_last_year_df

,permno,t,be_last_year
221708,10000.0,1987,0.418
221596,10001.0,1987,7.037
221597,10001.0,1988,7.038
221598,10001.0,1989,7.286
221599,10001.0,1990,8.466
...,...,...,...
392752,93435.0,2011,83.738
392753,93435.0,2012,119.323
427439,93436.0,2011,207.048
427440,93436.0,2012,224.045


In [37]:
compustat_be_last_year_df.shape # NOTE: There were 213229 observations read from the data set WORK.BE_LAST_YEAR.

(213229, 3)

In [38]:
# data ME_BM; 
#  merge ME_Jun (in = a) BE_last_year (in = b) ME_last_Dec (in = c); ** permno t ME_Jun ME_last_Dec BE_last_year ;
#  ** ME_Jun은 올해 6월, ME_last_Dec, BE_last_year은 작년 ;
#  by permno t; 
#  if a & b & c; 

ME_BM_df = pd.merge(
    left=CRSP_ME_JUN_df, 
    right=CRSP_ME_LAST_DEC_df,
    how='inner',
    on=['PERMNO', 'T'],
)

ME_BM_df = pd.merge(
    left=ME_BM_df,
    right=compustat_be_last_year_df,
    how='inner',
    left_on=['PERMNO', 'T'],
    right_on=['permno', 't'],
)

In [39]:
#  BM = BE_last_year/ME_last_Dec; 
#  keep permno t ME_Jun BM; 
# run;

ME_BM_df['BM'] = ME_BM_df['be_last_year'] / ME_BM_df['ME_LAST_DEC']
ME_BM_df = ME_BM_df[['PERMNO', 'T', 'ME_JUN', 'BM']]

In [40]:
ME_BM_df

,PERMNO,T,ME_JUN,BM
0,10000,1987,0.851594,0.210944
1,10001,1987,5.822125,1.014415
2,10001,1988,6.200000,1.207618
3,10001,1989,7.007000,1.145192
4,10001,1990,10.052250,0.818149
...,...,...,...,...
174164,93434,2011,24.998000,0.796865
174165,93434,2012,36.358170,0.569317
174166,93435,2011,28.538400,0.540183
174167,93436,2011,3028.966530,0.081921


In [41]:
ME_BM_df.shape # NOTE: The data set WORK.ME_BM has 174169 observations and 4 variables.

(174169, 4)